## The Language

The standard foundations of Mathematics is a **formal theory** consisting of the following. 

1. Two **alphabets**, one for **special symbols** like "&forall;" and "&exist;" and another for **variable** names like "x", "y", "z".
2. Syntax rules for excluding meaningless sentences like "∀ x (y:z) ∀ ∀ ∃", and for defining **free variables** and closed formulas without free variables. 
3. **Deduction rules** for constructing closed formulas from closed formulas.
4. A collection of closed formulas called **axioms**.

The axioms are meant to be "true" intuitively so that &forall; is supposed to behave like what we think of as "for all", etc.  For example, the Zermelo-Fraenkel set theory with the Axiom of Choice (ZFC) has ten axioms.  This is the most often cited foundation of Mathematics meaning that mathematical results should be reducible, in principle, to valid deductions in ZFC. 

Coda is meant to be an alternative to this as a foundation.  Rather than assuming or using predicate logic syntax, the foundation of Coda is **finite sequence**.  From this, one defines "pure" data and coda.  Logic in is then internal, derived and "2+ valued", where `()` and `(:)` are the iconic "true" and "false" valuea and where there are many "undecided" data values like `(foo:bar)`.  Coda has only one axiom - it is the axiom that defines what constitutes a valid definition.  The Coda language itself is merely one definition like any other.  This means that we don't need any additional axioms or deduction rules. 

In Coda, a definition is just a partial function from codas to data.  The Coda language, for instance, is a partial function from coda to data, acting on codas of the form ({...some source code...} A : B) where A and B are any data.  There are essentially only two operations in coda:

1. Concatenation of data A and data B, written `A B`.
2. Creating a coda from data A and data B, written `A:B`.

and the essential idea of the language is to make these two operations specifyable in text string source code ({...some source code...} A : B) and to get programmatic access to the "A" data (sometimes called "argument data") and the "B" data (sometimes called "input").  Thus, a sketch of the internal definition is:

* `{x : y} A : B` &rightarrow; `({x} A : B) : ({y} A : B)` ..to specify forming a coda from two data 
* `{x   y} A : B` &rightarrow; `({x} A : B)   ({y} A : B)` ..to specify concatenation of two data 
* `{A} A : B` &rightarrow; `A` ..to get the "A part" of the coda.
* `{B} A : B` &rightarrow; `B` ..to get the "B part" of the coda.

Because the compiler is merely a standard definition, the meaning of the language does not need a special explanation or additional axioms.  We don't even have to define proper syntax.  Coda has the unusual property that all finite sequences of bytes are valid language expressions.  There is no such thing as a syntax error. 

The full compiler is tiny, easily readable, and can be found in Language.py.  The full compiler and parser is only about 100 lines of Python.

In [2]:
#
#   Coda uses spaces to denote concatenation and parenthesis to group operations.
#   Comment lines must start with # are considered to be comments.
#
a b (c d) e (f g)

a b c d e f g

In [3]:
#
#   There are basically two operations: concatenation and colon. here, 
#   for instance, we reverse the order of 1 2 3 using rev.  This works by first 
#   creating the coda (rev : 1 2 3).  A definition applies to this coda, which 
#   results the "input" to rev in reverse order 3 2 1.  
#
rev : 1 2 3 

3 2 1

In [4]:
#
#   Some "commands" like rev have arguments.  first, for example, gets 
#   the first items from a sequence. 
#
first 2 : a b c d e 

a b

In [5]:
#
#   Colons can be combined. The default grouping is from the right.  This is first 2 : (rev : a b c d e).
#   (first 2 : rev ) : a b c d e is completely different.  Intuitively, you can think of data flowing 
#   from the right to the left with a b c d e consumed by rev, and then, the result gets consumed 
#   by "first 2:".
#
first 2 : rev : a b c d e 

e d

In [6]:
#
#   Lines are continued by indentation.
#
first 2 : rev : a b 
   c d e

e d

In [7]:
#
#   Lines without indentation are the same as (line1) (line2), etc.  
#
first 2 : rev : a b
c d e

b a c d e

In [8]:
#
#   Text in curly braces is interpreted as Coda language expressions.  "B" in the language, is a special symbol 
#   which refers to the "input", in this case, the data a b c d e... 
#
{first 2 : B} : a b c d e 

a b

In [9]:
#
#   The general language coda is ({...} A : B)  where "A" in the language referes to A and "B" refers to B.
#
{A B A B} 1 2 : x y 

1 2 x y 1 2 x y

In [10]:
#
#   So you can do...
#
{first A : B} 2 : a b c d e 

a b

In [11]:
#
#   You can use "step" to show how this get's evaluated in more detail.  
#   Each step below is made by application definitions.  You can see 
#   that Coda freely mixes data and source code as it computes.  The compiler 
#   is just a definition like any other.
#
step : {first A : B} 2 : a b c d e

[0] (({{first A : B} 2 }:):({ a b c d e}:))
[1] (({{first A : B} 2}:):({a b c d e}:))
[2] (({{first A : B}}:) ({2}:):({a}:) ({b c d e}:))
[3] (({first A } 2:a ({b}:) ({c d e}:)):({ B} 2:a ({b}:) ({c d e}:)))
[4] (({first A} 2:a b ({c}:) ({d e}:)):({B} 2:a b ({c}:) ({d e}:)))
[5] (({first} 2:a b c ({d}:) ({e}:)) ({A} 2:a b c ({d}:) ({e}:)):a b c ({d}:) ({e}:))
[6] a (first 1:b c d e)
[7] a b (first 0:c d e)
[8] a b

In [12]:
#
#   You can use "def" to make your own definitions.  This one defines a new 
#   command "mydef" which gets the first n elements from the reversed input. 
#
def mydef : {first A : rev : B} 
mydef 2 : a b c d e

e d

In [13]:
#
#   Notice how Coda handles "undecided" data like (foo:bar) where 
#   no current definition acts on (foo:...)...
#
rev : 1 2 3 (foo:bar)

(rev:(foo:bar)) 3 2 1

In [14]:
#
#   (foo:bar) is treated like a "variable", i.e. like something 
#   that may have a value in the future.  If (foo:bar) gets defined later, 
#   then (rev:(foo:bar)) 3 2 1 will evaluate to the correct answer while, at the same 
#   time, current computations will be correct, for instance... 
#
last 2 : rev : 1 2 3 (foo:bar)

2 1

In [ ]:
#
#    Sequences in Coda are always finite sequences, but this isn't 
#    a real limitation.  The natural numbers, for instance are embodied by 
#    a definition (nat:n) -> n (nat:n+1).  This means that (nat:0) will get 
#    evaluated as necessary.  Although this is finite,  
#
(nat:0)

In [ ]:
#
#    What's the "reverse of the natural numbers?".  Let's see... 
#
rev : nat : 0

In [ ]:
#
#...And, for example, you can reverse twice to get the original natural numbers back.
#
rev : rev : nat : 0 

In [ ]:
#
#   Notice, that, as expected, the "first element of the reverse of the 
#   natural numbers" is not defined.  Yet, the last element is perfectly OK.
#
last : rev : nat : 0 

In [ ]:
#
#   defs: produces a list of current definitions in context and 
#   help : first, etc. gives help information and demos which 
#   you can execute by hand or via the "demo" command.
#
help : first

In [ ]:
demo 4 : first 

### Combinatorics 

Any system of reasoning must contain undefined terms.  In the case of Coda, the **finite sequence** is the foundational concept that is assumed to be understood without definition.  It's pleasing and expected, then, that the low level operations in Coda are things are the natural low level combinatorial operations on finite sequences.  We've looked at two of these as simple examples:

* `rev` reverses the order of a sequence. 
* `first` gets the first n items of a sequence. 

Some of the most important operations are combinatorics of the form (name A:B) where A gets used in various combinatoric ways on B.  The most important one of these is called `ap`.  The idea is that something like this 

* `ap A : b1 b2 b3...bn`

is defined to be "A applied to each b1,...bn".  It's defined to be (A:b1) (A:b2)...(A:bn).  The actual definition of ap is 

* `ap A : B C -> (ap A:B) (ap A:C)`
* `ap A : B -> (A:B)` if B is an atom 
* `ap A : () -> ()` 

which is the same idea, applicable to all "input" data (B) and all "arguments" (A).  Coda has a collection of these named definitions built into the system.  They are all simple combinations of the three ingredients``(name A : B)`.  These are basic definitions underlying many of the more familiar mathematical objects built on top.  As far as I know, these don't have conventional names, so we are forced into naming them.

| coda | schematic result |
|------|------------|
|  `ap` A:b1 b2 b3  | (A:b1) (A:b2) (A:b3) |
|  `app` a1 a2 a3:B | (a1:B) (a2:B) (a3:B) |
| `ap2` a a1 a2 a3:B | (a a1:b1) (a a2:b2) (a a3:b3)  |
| `apbin` A:b1 b2 b3 | (A b1 : (A b2 : (A b3 : b4) ) )  |
| `apby` 2 A:b1 b2 b3 b4... | (A:b1 b2) (A:b2 b3)... |
| `apif` A:b1 b2... |  (b1..if (A:b1)) (b2..if (A:b2))...| 

We will do a few examples below.  Each has a help entry and examples which can be run in the notebook or command line.  Although the names are not familiar, they are so simple that an example or two is an effective way to learn them. 

In [ ]:
#
#   Each has a help entry with examples.
#   You can run the demos by hand or with the demo command. 
#
help : ap

In [ ]:
ap {foo : B} : 1 2 3 

There are several variations on `ap` 
* `app` turns a binary operation into it's sequential version.
* `apx` applies all arguments to all inputs. 


In [ ]:
foo : 1 2 3 4 

In [ ]:
ap foo : a b c

In [ ]:
app a b c : 1 2 3 

In [ ]:
ap app a b c : 1 2 3 

In [ ]:
ap2 a b c : 1 2 3 

In [ ]:
ap2 nth1 3 4 : a b c d 

In [ ]:
ap ap2 a b c : 1 2 3 

In [ ]:
apbin foo : a b c d 

In [ ]:
#
#   apbin is typically used to turn a binary associative operation 
#   into the equivalent operation on a sequence.  So, for instance, 
#   (int_add A:B) is the low level binary operation adding A and B as 
#   integers...
#
int_add 3 : 5

In [ ]:
#
#...and apbin turns this into a sequence sum of any number of values.
#
apbin int_add : 1 2 3 4 5

In [ ]:
apby 2 foo : a b c d e f g

In [ ]:
apif {(count:get bin:B)=2} : (bin:a b) (bin:a b c) (bin:x y) (bin:a b c d)

In [ ]:
#
#   A few dozen definitions are built into the system and enter via python modules. 
#   the rest come from .co coda source code files.
#
defs:

In [ ]:
#
#   You can also assign a data values to any undefined coda.  For convenience, 
#   the compiler takes x? and makes this actually (?:x) to make convenient things that 
#   look like "variables" but are actually codas.  
#
let x? : 1 2 3 
x?